In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso 
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor 
from catboost import CatBoostRegressor
import warnings
 

C:\Users\akram\AppData\Local\Temp\ipykernel_4352\2012916091.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df =  pd.read_csv('Data/cleaned_data.csv')
df.head()

,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,gender,math_score,reading_score,writing_score,Total_score,Avrage
0,group B,bachelor's degree,standard,none,female,72,72,74,218,72.67
1,group C,some college,standard,completed,female,69,90,88,247,82.33
2,group B,master's degree,standard,none,female,90,95,93,278,92.67
3,group A,associate's degree,free/reduced,none,male,47,57,44,148,49.33
4,group C,some college,standard,none,male,76,78,75,229,76.33


**Preparing X And Y variabls** 

In [3]:
X = df.drop(columns=['Avrage','Total_score'],axis=1)
X.sample(5)

,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,gender,math_score,reading_score,writing_score
397,group C,associate's degree,standard,none,female,85,89,95
489,group A,associate's degree,free/reduced,completed,male,79,82,82
769,group A,some college,free/reduced,none,male,58,60,57
906,group B,high school,standard,none,male,57,48,51
788,group C,associate's degree,free/reduced,none,male,68,65,61


In [4]:
X.shape

(1000, 8)

In [5]:
y =  df['Avrage']
y

0      72.67
1      82.33
2      92.67
3      49.33
4      76.33
       ...  
995    94.00
996    57.33
997    65.00
998    74.33
999    83.00
Name: Avrage, Length: 1000, dtype: float64

In [6]:
print('Categories in gender variable:',end=" ")
print(df['gender'].unique())

print('Categories in race/Ethnicity variable:',end=" ")
print(df['race_ethnicity'].unique())
print('')
print('Categories in paretale level of education variable:',end=" ")
print(df['parental_level_of_education'].unique())
print('')
print('Categories in lunch variable:',end=" ")
print(df['lunch'].unique())
print('Categories in test preparation course variable:',end=" ")
print(df['test_preparation_course'].unique())


Categories in gender variable: ['female' 'male']
Categories in race/Ethnicity variable: ['group B' 'group C' 'group A' 'group D' 'group E']

Categories in paretale level of education variable: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']

Categories in lunch variable: ['standard' 'free/reduced']
Categories in test preparation course variable: ['none' 'completed']


**standradtion and encoding**

In [7]:
num_features = X.select_dtypes(exclude='O').columns
num_features

Index(['math_score', 'reading_score', 'writing_score'], dtype='object')

In [8]:
catfeatures = X.select_dtypes(include='O').columns
catfeatures

Index(['race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course', 'gender'],
      dtype='object')

In [9]:
#using ColumnTransformer for apply multiple transforms on multiplle column 
#oneHotencoder for categorical features m and  standerdScaeler for numric features  
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
ohe = OneHotEncoder()

preprocesseur =ColumnTransformer(

    [   ("OneHotEncoder",ohe,catfeatures),
        ('StandrScaler',numeric_transformer,num_features),
    
    ]
)

In [10]:
X =preprocesseur.fit_transform(X)

In [11]:
X

array([[ 0.        ,  1.        ,  0.        , ...,  0.39002351,
         0.19399858,  0.39149181],
       [ 0.        ,  0.        ,  1.        , ...,  0.19207553,
         1.42747598,  1.31326868],
       [ 0.        ,  1.        ,  0.        , ...,  1.57771141,
         1.77010859,  1.64247471],
       ...,
       [ 0.        ,  0.        ,  1.        , ..., -0.46775108,
         0.12547206, -0.20107904],
       [ 0.        ,  0.        ,  0.        , ...,  0.12609287,
         0.60515772,  0.58901542],
       [ 0.        ,  0.        ,  0.        , ...,  0.71993682,
         1.15336989,  1.18158627]])

In [12]:
X.shape

(1000, 20)

**Split th Data into train and test**

In [95]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape,

((800, 20), (200, 20), (800,))

**Aply multiple models**

In [32]:
models = {
    'LinearRegression':LinearRegression(),
    'Ridge':Ridge(),
    'Lasso':Lasso(),
    'K-Neighbors Regressor':KNeighborsRegressor(),
    'XGB Regressor':XGBRegressor(),
    'CatBoost Regressor':CatBoostRegressor(verbose=False),
    'Decision Tree Regressor:':DecisionTreeRegressor(),
    'RandomForestRegressor':RandomForestRegressor(),
    #'Support Vector machine':SVR
}

In [33]:
def Model_evaluate(true_val,predicted_val): 
    MAE = mean_absolute_error(true_val,predicted_val)
    MSE = mean_squared_error(true_val,predicted_val)
    rmse = np.sqrt(mean_squared_error(true_val,predicted_val))
    r2score = r2_score(true_val,predicted_val)

    return MAE,r2score,MSE,rmse

In [55]:
model_list =[]
r2_score_list = [] 
for mod in range (len(list(models))):
    model = list(models.values())[mod]
    model.fit(X_train,y_train)
    
    #make predictions: 
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    #Evaluate model 
    model_train_MAE, model_train_r2score, model_train_MSE,model_train_RMSE  = Model_evaluate(y_train,y_train_pred)    
    model_test_MAE, model_test_r2score, model_test_MSE,model_test_RMSE = Model_evaluate(y_test,y_test_pred)  

    #Print information 

    print('\033[1m'+list(models.keys())[mod]+'\033[0m') 
    model_list.append(list(models.keys())[mod])
    print(model_list)
    print('MODEL PERFORMANCE FOR TRAINING SET')
    print("--Mean Squared Error: {:.4f}".format(model_train_MSE))
    print("--Root Mean Squared Error: {:.4f}".format(model_train_RMSE))
    print("--Mean Absolute Error: {:.4f}".format(model_train_MAE))
    print("--R2 Score: {:.4f}".format(model_train_r2score)) 

    print("----------------------------------------------------------")

    print('MODEL PERFORMANCE FOR Test SET')
    print("--Mean Squared Error: {:.4f}".format(model_test_MSE))
    print("--Root Mean Squared Error: {:.4f}".format(model_test_RMSE))
    print("--Mean Absolute Error: {:.4f}".format(model_test_MAE))
    print("--R2 Score: {:.4f}".format(model_test_r2score)) 

    r2_score_list.append(model_test_r2score)
    

    print("="*30)
    print('\n')     

LinearRegression
['LinearRegression']
MODEL PERFORMANCE FOR TRAINING SET
--Mean Squared Error: 0.0000
--Root Mean Squared Error: 0.0027
--Mean Absolute Error: 0.0023
--R2 Score: 1.0000
----------------------------------------------------------
MODEL PERFORMANCE FOR Test SET
--Mean Squared Error: 0.0000
--Root Mean Squared Error: 0.0026
--Mean Absolute Error: 0.0022
--R2 Score: 1.0000


Ridge
['LinearRegression', 'Ridge']
MODEL PERFORMANCE FOR TRAINING SET
--Mean Squared Error: 0.0001
--Root Mean Squared Error: 0.0085
--Mean Absolute Error: 0.0068
--R2 Score: 1.0000
----------------------------------------------------------
MODEL PERFORMANCE FOR Test SET
--Mean Squared Error: 0.0001
--Root Mean Squared Error: 0.0094
--Mean Absolute Error: 0.0073
--R2 Score: 1.0000


Lasso
['LinearRegression', 'Ridge', 'Lasso']
MODEL PERFORMANCE FOR TRAINING SET
--Mean Squared Error: 1.1332
--Root Mean Squared Error: 1.0645
--Mean Absolute Error: 0.8505
--R2 Score: 0.9943
--------------------------------

In [49]:
list(zip(model_list,r2_score_list))

[('RandomForestRegressor', 0.9935634145479869)]

# **Results**

In [64]:
results = pd.DataFrame(list(zip(model_list,r2_score_list)),columns=['MODEL','R2 Score']).round(2).sort_values(by=['R2 Score'],ascending=False)
results

,MODEL,R2 Score
0,LinearRegression,1.00
1,Ridge,1.00
2,Lasso,0.99
4,XGB Regressor,0.99
5,CatBoost Regressor,0.99
6,Decision Tree Regressor:,0.99
7,RandomForestRegressor,0.99
3,K-Neighbors Regressor,0.96


* the Right model:</br>
**Linear rigression**

In [75]:
lin_model = LinearRegression(fit_intercept=True)
lin_model.fit(X_train,y_train)
y_pred =lin_model.predict(X_test)
score =r2_score(y_test,y_pred)*100
print('Accuracy of the model is: %.1f'%score) 


Accuracy of the model is: 100.0


**Predicted values and the true values** 


In [83]:
show_values = pd.DataFrame({"Actuale Values":y_test,"Predicted Values":y_pred,"Diffrance":(y_test-y_pred)})
show_values

,Actuale Values,Predicted Values,Diffrance
521,87.00,87.000267,-0.000267
737,64.00,63.999725,0.000275
740,75.00,75.000526,-0.000526
660,74.67,74.665718,0.004282
411,81.67,81.665749,0.004251
...,...,...,...
408,55.00,55.001152,-0.001152
332,57.00,56.999817,0.000183
208,77.00,76.999725,0.000275
613,72.00,71.999794,0.000206
